# Test submitting jobs to a running Ray cluster

## Setup

In [ ]:
!pip install ray

### Use Kube service corresponding to the Ray cluster

In [ ]:
RAY_DASHBOARD = "http://raycluster-codeflare-head-svc:8265"
SCRIPT = "hello_ray.py"

## Testing job submission

In [ ]:
from ray.job_submission import JobSubmissionClient

# If using a remote cluster, replace 127.0.0.1 with the head node's IP address.
client = JobSubmissionClient(RAY_DASHBOARD)
job_id = client.submit_job(
    # Entrypoint shell command to execute
    entrypoint=f"python {SCRIPT}",
    # Path to the local directory that contains the script.py file
    runtime_env={"working_dir": "./"}
)
print(job_id)

In [ ]:
from ray.job_submission import JobSubmissionClient, JobStatus
import time

# If using a remote cluster, replace 127.0.0.1 with the head node's IP address.
client = JobSubmissionClient(RAY_DASHBOARD)
job_id = client.submit_job(
    # Entrypoint shell command to execute
    entrypoint=f"python {SCRIPT}",
    # Path to the local directory that contains the script.py file
    runtime_env={"working_dir": "./"}
)
print(job_id)

def wait_until_status(job_id, status_to_wait_for, timeout_seconds=5):
    start = time.time()
    while time.time() - start <= timeout_seconds:
        status = client.get_job_status(job_id)
        print(f"status: {status}")
        if status in status_to_wait_for:
            break
        time.sleep(1)


wait_until_status(job_id, {JobStatus.SUCCEEDED, JobStatus.STOPPED, JobStatus.FAILED})
logs = client.get_job_logs(job_id)
print(logs)

## Interacting with Long-running Jobs

In [ ]:
job_id = client.submit_job(
    # Entrypoint shell command to execute
    entrypoint="python -c 'import time; print(\"Sleeping...\"); time.sleep(60)'"
)
wait_until_status(job_id, {JobStatus.RUNNING})
print(f'Stopping job {job_id}')
client.stop_job(job_id)
wait_until_status(job_id, {JobStatus.SUCCEEDED, JobStatus.STOPPED, JobStatus.FAILED})
logs = client.get_job_logs(job_id)
print(logs)

## Dependency Management

In [ ]:
job_id = client.submit_job(
    # Entrypoint shell command to execute
    entrypoint=f"python {SCRIPT}",
    # Runtime environment for the job, specifying a working directory and pip package
    runtime_env={
        "working_dir": "./",
        "pip": ["requests==2.26.0"]
    }
)